In [14]:
import torch
import torch.nn as nn
import numpy as np
from torch.nn.utils import clip_grad_norm_ #!!

### 1.setting 

In [25]:
#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = torch.device('cpu')

In [26]:
device

device(type='cpu')

In [27]:
embed_size = 128   # dimension of word vector
hidden_size = 1024 #feature
num_layers =1
num_epochs =5
num_samples =1000
batch_size =20
seq_length =30
learning_rate =0.002

### 2. Corpus

In [28]:
class Dictionary(object):
    def __init__(self):
        self.word2idx= {}
        self.idx2word= {}
        self.idx = 0;
        
    def add_word(self, word):
        if not word in self.word2idx:
            self.word2idx[word] = self.idx
            self.idx2word[self.idx] = word
            self.idx+=1
            
    def __len__(self):
        return len(self.word2idx)

class Corpus(object):
    def __init__(self):
        self.dictionary = Dictionary() #elem dictionary init as Dictionary object
    
    def get_data(self, file_path, batch_size=20):
        #First: add words into the dictionary
        with open(file_path, 'r') as file:
            num_tokens =0 
            # every line
            for line in file:
                words = line.split() +['<eos>'] #end of sentence
                num_tokens += len(words)
                #every word
                for word in words:
                    self.dictionary.add_word(word)
        
        #Second: fenci tokenize
        ids = torch.LongTensor(num_tokens) #64 bit interger size: num_tokens
        num_token = 0
        with open(file_path,'r') as file:
            for line in file:
                words = line.split() + ['<eos>']
                for word in words:
                    ids[num_token] = self.dictionary.word2idx[word]
                    num_token +=1
                    
        num_batches = ids.size(0) // batch_size
        ids = ids[:num_batches * batch_size]
        return ids.view(batch_size, -1)

### 3. input data

In [29]:
#input the data
#init as Corpus object
corpus = Corpus()
ids = corpus.get_data('data/train.txt',batch_size)
vocab_size =len(corpus.dictionary)
num_batches = ids.size(1) //seq_length

In [21]:
ids.shape # ids.view(batch_size,-1) # the size -1 is inferred from other dimensions

torch.Size([20, 46479])

In [22]:
vocab_size

10000

In [23]:
ids.view(20,-1)[2]

tensor([  27,  930,   42,  ...,  392, 4864,   26])

### 4. Language Model

In [30]:
class RNNLM(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size, num_layers):
        super(RNNLM, self).__init__()#inheritage
        self.embed = nn.Embedding(vocab_size, embed_size)
        self.lstm = nn.LSTM(embed_size, hidden_size, num_layers,batch_first=True)
        self.linear = nn.Linear(hidden_size, vocab_size)
    
    def forward(self, x, h):
        #embedding
        x = self.embed(x)
        
        out, (h, c) = self.lstm(x, h)
        
        #(batch_size * seq_len, hidden_size)
        out = out.reshape(out.size(0) * out.size(1) , out.size(2))
        
        out = self.linear(out)
        return out, (h,c)

model = RNNLM(vocab_size, embed_size, hidden_size, num_layers).to(device)

In [31]:
# loss and optimize
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)

In [32]:
ids.size(1) - seq_length # from head to tail sentence

46449

In [ ]:
#backward
def detach(states):
    return [state.detach() for state in states]

#train

for epoch in range(num_epochs):
    #init
    states = (torch.zeros(num_layers, batch_size, hidden_size).to(device),
             torch.zeros(num_layers, batch_size, hidden_size).to(device))
    
    for i in range(0, ids.size(1) - seq_length, seq_length):
        #every mini-batch
        inputs = ids[:, i: i+seq_length].to(device) #[ , one seq]
        targets = ids[:,(i+1):(i+1)+seq_length].to(device)
        
        #forward
        states = detach(states)
        outputs,states  = model(inputs, states)
        loss = criterion(outputs,targets.reshape(-1)) # (h - y) yi chuan!!! sum
        
        #optimize
        
        model.zero_grad()
        loss.backward()
        clip_grad_norm_(model.parameters(),0.5)
        optimizer.step()
        
        step = (i+1)// seq_length
        if step %100 ==0:
            print('迭代到第[{}/{}]轮，step [{}/{}], Loss{:.4f}, 困惑度: {:5.2f}'
                  .format(epoch+1,num_epochs,step,num_batches, loss.item(), np.exp(loss.item())))


迭代到第[1/5]轮，step [0/1549], Loss9.2126, 困惑度: 10022.68
迭代到第[1/5]轮，step [100/1549], Loss5.9987, 困惑度: 402.89
迭代到第[1/5]轮，step [200/1549], Loss5.9280, 困惑度: 375.39
迭代到第[1/5]轮，step [300/1549], Loss5.7989, 困惑度: 329.94
迭代到第[1/5]轮，step [400/1549], Loss5.6805, 困惑度: 293.09
迭代到第[1/5]轮，step [500/1549], Loss5.1068, 困惑度: 165.15
迭代到第[1/5]轮，step [600/1549], Loss5.2065, 困惑度: 182.46
迭代到第[1/5]轮，step [700/1549], Loss5.3672, 困惑度: 214.25
迭代到第[1/5]轮，step [800/1549], Loss5.2160, 困惑度: 184.19
迭代到第[1/5]轮，step [900/1549], Loss5.0603, 困惑度: 157.64
迭代到第[1/5]轮，step [1000/1549], Loss5.1421, 困惑度: 171.07
迭代到第[1/5]轮，step [1100/1549], Loss5.3135, 困惑度: 203.05
迭代到第[1/5]轮，step [1200/1549], Loss5.1753, 困惑度: 176.85
迭代到第[1/5]轮，step [1300/1549], Loss5.0651, 困惑度: 158.40
迭代到第[1/5]轮，step [1400/1549], Loss4.8805, 困惑度: 131.69
迭代到第[1/5]轮，step [1500/1549], Loss5.1535, 困惑度: 173.03
迭代到第[2/5]轮，step [0/1549], Loss5.4517, 困惑度: 233.16
迭代到第[2/5]轮，step [100/1549], Loss4.4988, 困惑度: 89.91
迭代到第[2/5]轮，step [200/1549], Loss4.6177, 困惑度: 101.26
迭代到第[2/5]

In [ ]:
#test


with torch.no_grad():
    with open('sample.txt', 'w') as f:

        state = (torch.zeros(num_layers, 1, hidden_size).to(device),
                 torch.zeros(num_layers, 1, hidden_size).to(device))

        #random input
        prob = torch.ones(vocab_size)
        input = torch.multinomial(prob, num_samples=1).unsqueeze(1).to(device)

        for i in range(num_samples):
            # forward
            output, state = model(input, state)

            # predic
            prob = output.exp()
            word_id = torch.multinomial(prob, num_samples=1).item()

            # result as next input
            input.fill_(word_id)

            #out
            word = corpus.dictionary.idx2word[word_id]
            word = '\n' if word == '<eos>' else word + ' '
            f.write(word)

            if (i+1) % 100 == 0:
                print('生成了 [{}/{}] 个词，存储到 {}'.format(i+1, num_samples, 'sample.txt'))

# checkpoints
torch.save(model.state_dict(), 'model.ckpt')